# Advent of Code - Day 8 Year 2024

In [1]:
DAY = 8
YEAR = 2024
#######
import json
import sys 
sys.path.append('../../')
from utils import get_pb_data, rephrase_pb, open_link, render_json
link, data, exmp, ae1, ae2, exmps, out = get_pb_data(DAY, YEAR)

print(f'problem link: {link}')

nb examples: 1
problem link: https://adventofcode.com/2024/day/8


In [2]:
# with open("./_puzzle_data.json", "r") as f: inp = json.loads(f.read())
# render_json(inp)

In [3]:
await rephrase_pb(DAY, YEAR, 1)

**Problem Rephrasing:**

Find the total number of unique antinode locations on a 2D map containing antennas. 

**Key Rules:**
- Antennas are represented by lowercase letters, uppercase letters, or digits on a grid
- Only antennas with the **same frequency** (exact same character) create antinodes
- An antinode appears when two same-frequency antennas are aligned, with one being **twice** as far from the antinode as the other
- Each pair of same-frequency antennas creates **two** antinodes
- Antinodes can overlap with antenna positions
- Different frequencies (like 'A' and 'a') don't interact

**Input Example:**
```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

**Goal:** Count how many unique positions within the map contain at least one antinode.

- frequencies can be represented by: lowercase, uppercase or a digit.

| "However, antinodes can occur at locations that contain antennas"

- I probably need a separate grid to add the antinodes

- You could have antinodes outside the bounds of the grid, in that case they don't count



- where do antinodes occur ? 

In particular, an antinode occurs at any point that is **perfectly in line** with two antennas of the same frequency - but only **when one of the antennas is twice as far away as the other**.

- "aligned": can be horizontal, vertical, diagonal. Although all the cases in the example are diagonally aligned

First idea:

- get the location of each antenna
- consider each possible couple of antennas
- for each determine the possible positions of antinodes. If they are within the bound of the grid, increment the count of antinodes

Is this computationally feasible ? Let's calculate the number of combinations to consider in total

first, how many combinations to consider for one specific antenna frequency

if there is only 1 of this type of antenna --> no antinodes

If there are (n) with n >= 2, there are C(n, 2) ("n choose k") 

asking claude how to calculate combinations in python:

In [11]:
from math import comb 

n, k = 1, 2
print(n , k, comb(n, k))

n, k = 2, 2
print(n , k, comb(n, k))

n, k = 4, 2
print(n , k, comb(n, k))

1 2 0
2 2 1
4 2 6


In [14]:
dat = exmp 
print(exmp)

............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [16]:
from collections import Counter 

a_cnt = Counter([c for c in dat if c not in ('.', '\n')])
a_cnt

Counter({'0': 4, 'A': 3})

In [20]:
for a in a_cnt:
    print(a, a_cnt[a])

0 4
A 3


In [21]:
sum([comb(a_cnt[a], 2) for a in a_cnt])

9

In [22]:
def calc_nb_comb(dat: str) -> int:
    ...

In [23]:
def calc_nb_comb(dat: str) -> int:
    a_cnt = Counter([c for c in dat if c not in ('.', '\n')])
    return sum([comb(a_cnt[a], 2) for a in a_cnt])

In [25]:
dat = exmp
calc_nb_comb(dat)

9

In [26]:
dat = data
calc_nb_comb(dat)

295

that's reasonably low, I think I can just consider all possible combinations to solve the problem

Let's say I have a particular couple of antennas, with their position on the grid

In [27]:
print(exmp)

............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [32]:
dat = exmp
grid = dat.splitlines()
grid

['............',
 '........0...',
 '.....0......',
 '.......0....',
 '....0.......',
 '......A.....',
 '............',
 '............',
 '........A...',
 '.........A..',
 '............',
 '............']

In [33]:
len(grid[0])

12

In [36]:
## first couple of 0
x1, y1 = (8,1)
x2, y2 = (5,2)
a1 = (x1, y1)
a2 = (x2, y2)
w, h = len(grid[0]), len(grid)
## check
print(grid[y1][x1])
print(grid[y2][x2])

0
0


In [38]:
Coord = tuple[int, int]
def get_coord_antinodes(a1: Coord, a2: Coord) -> list[Coord]:
    ...

3 possibilities:

- antennas are vertically aligned: occurs when x1 = x2
- antennas are horizontally aligned: occurs when y1 = y2
- antennas are diagonal to each other: otherwise

actually I don't need to differentiate the possibilities, I can simply calculate the coordinate delta

In [44]:
x1, y1 = (2,1)
x2, y2 = (2,2)
d = (max(x1, x2) - min(x1, x2), max(y1, y2) - min(y1, y2))
d

(0, 1)

In [51]:
cnt = 0
x1, y1 = (2,1)
x2, y2 = (2,4)
xd, yd = (max(x1, x2) - min(x1, x2), max(y1, y2) - min(y1, y2))
d = xd, yd
d

(0, 3)

In [54]:
# let's organize the antennas by ascending y, then ascending x
ants = [(x1, y1), (x2, y2)]
ants = sorted(ants, key=lambda x: (x[1], x[0]))
an_coord = [(ants[0][0]-d[0],ants[0][1]-d[1]), (ants[1][0]+d[0],ants[1][1]+d[1])]
an_coord 

[(2, -2), (2, 7)]

In [55]:
an_coord = [(x,y) for x,y in an_coord if x >=0 and x < w and y >= 0 and y < h]
an_coord

[(2, 7)]

In [ ]:
len(an_coord)

In [61]:
def get_coord_antinodes(a1: Coord, a2: Coord) -> list[Coord]:
    x1, y1 = a1
    x2, y2 = a2
    xd, yd = (max(x1, x2) - min(x1, x2), max(y1, y2) - min(y1, y2))
    anten = [a1, a2]
    anten = sorted(anten, key=lambda x: (x[1], x[0]))
    anode = [(anten[0][0]-xd,anten[0][1]-yd), (anten[1][0]+xd,anten[1][1]+yd)]
    anode = [(x,y) for x,y in anode if x >=0 and x < w and y >= 0 and y < h]
    return anode

In [66]:
all_anode = set()

In [75]:
def format_input(dat: str) -> list[tuple[str, Coord]]:
    ...

In [93]:
dat = exmp
grid = dat.splitlines()
grid

['............',
 '........0...',
 '.....0......',
 '.......0....',
 '....0.......',
 '......A.....',
 '............',
 '............',
 '........A...',
 '.........A..',
 '............',
 '............']

In [94]:
all_antennas = [(c,(x,y)) for y,l in enumerate(grid) for x,c in enumerate(l) if c not in ('.', '\n')]
all_antennas

[('0', (8, 1)),
 ('0', (5, 2)),
 ('0', (7, 3)),
 ('0', (4, 4)),
 ('A', (6, 5)),
 ('A', (8, 8)),
 ('A', (9, 9))]

In [99]:
antenna_types = set([c for c,_ in all_antennas])
antenna_types

{'0', 'A'}

In [100]:
[(a, [co for c,co in all_antennas if c == a]) for a in antenna_types]

[('A', [(6, 5), (8, 8), (9, 9)]), ('0', [(8, 1), (5, 2), (7, 3), (4, 4)])]

In [139]:
def format_input(dat: str) -> tuple[list[tuple[str, Coord]], list[str]]:
    grid = [[c for c in l] for l in dat.splitlines()]
    all_antennas = [(c,(x,y)) for y,l in enumerate(grid) for x,c in enumerate(l) if c not in ('.', '\n')]
    antenna_types = set([c for c,_ in all_antennas])
    out = [(a, [co for c,co in all_antennas if c == a]) for a in antenna_types]
    return out, grid

In [140]:
dat = exmp 
ants, grid = format_input(dat)
for e in ants: print(e)

('A', [(6, 5), (8, 8), (9, 9)])
('0', [(8, 1), (5, 2), (7, 3), (4, 4)])


In [141]:
at = ants[0]
at[1]

[(6, 5), (8, 8), (9, 9)]

claude: generate all possible combinations of k elements amongst a list of n in python

In [142]:
from itertools import combinations
list(combinations(at[1], 2))

[((6, 5), (8, 8)), ((6, 5), (9, 9)), ((8, 8), (9, 9))]

In [143]:
def generate_antenna_combinations(antenna_coords: list[Coord]) -> list[tuple[Coord, Coord]]:
    ...

In [144]:
def generate_antenna_combinations(antenna_coords: list[Coord]) -> list[tuple[Coord, Coord]]:
    from itertools import combinations
    return list(combinations(antenna_coords, 2))

In [145]:
dat = exmp 
ants, grid = format_input(dat)
combs = [cb for a in ants for cb in generate_antenna_combinations(a[1])]
for cb in combs: 
    print(cb)

((6, 5), (8, 8))
((6, 5), (9, 9))
((8, 8), (9, 9))
((8, 1), (5, 2))
((8, 1), (7, 3))
((8, 1), (4, 4))
((5, 2), (7, 3))
((5, 2), (4, 4))
((7, 3), (4, 4))


In [146]:
[co for cb in combs for co in get_coord_antinodes(cb[0], cb[1])]

[(4, 2),
 (10, 11),
 (3, 1),
 (7, 7),
 (10, 10),
 (5, 0),
 (8, 3),
 (8, 5),
 (8, 7),
 (3, 1),
 (9, 4),
 (4, 0),
 (5, 6),
 (4, 2),
 (7, 5)]

In [147]:
ans = set([co for cb in combs for co in get_coord_antinodes(cb[0], cb[1])])
ans

{(3, 1),
 (4, 0),
 (4, 2),
 (5, 0),
 (5, 6),
 (7, 5),
 (7, 7),
 (8, 3),
 (8, 5),
 (8, 7),
 (9, 4),
 (10, 10),
 (10, 11)}

In [148]:
_, grid2 = format_input(dat)
for x,y in ans:
    grid2[y][x] = "#"
grid2 = [str(i) for i in ] + 

[['.', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '#', '.', '.', '.', '.', '0', '.', '.', '.'],
 ['.', '.', '.', '.', '#', '0', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '0', '#', '.', '.', '.'],
 ['.', '.', '.', '.', '0', '.', '.', '.', '.', '#', '.', '.'],
 ['.', '.', '.', '.', '.', '.', 'A', '#', '#', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.']]

In [161]:
def display_grid(grid: list[list[str]]):
    w, h = len(grid[0]), len(grid)
    dd = []
    dl = [' '] + [str(x) for x in range(w)]
    dd.append(dl)
    for y,l in enumerate(grid):
        dl = [str(y)] + l
        dd.append(dl)    
    for l in dd:
        print(l)

In [163]:
print(exmp)

............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [164]:
def display_grid_2(grid: list[list[str]]):
    for l in grid:
        print(''.join(l))

In [165]:
display_grid_2(grid2)

....##......
...#....0...
....#0......
.......0#...
....0....#..
......A##...
.....#......
.......##...
........A...
.........A..
..........#.
..........#.


wrong. I went too fast

let's build things upa gain step by step

In [166]:
def format_input(dat: str) -> tuple[list[tuple[str, Coord]], list[str]]:
    grid = [[c for c in l] for l in dat.splitlines()]
    all_antennas = [(c,(x,y)) for y,l in enumerate(grid) for x,c in enumerate(l) if c not in ('.', '\n')]
    antenna_types = set([c for c,_ in all_antennas])
    out = [(a, [co for c,co in all_antennas if c == a]) for a in antenna_types]
    return out, grid

In [168]:
dat = exmp
an_coord, grid = format_input(dat)
an_coord

[('A', [(6, 5), (8, 8), (9, 9)]), ('0', [(8, 1), (5, 2), (7, 3), (4, 4)])]

In [179]:
def print_grid(an_coord, grid):
    w, h = len(grid[0]), len(grid)
    grid_ps = [['.' for _ in range(w)] for _ in range(h)]
    
    for atyp, coords in an_coord:
        for x,y in coords:
            grid_ps[y][x] = atyp
    for l in grid_ps:
        print(''.join(l))        

In [180]:
print_grid(an_coord, grid)

............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [188]:
def print_grid(an_coord, grid, antinode_coords = None):
    w, h = len(grid[0]), len(grid)
    grid_ps = [['.' for _ in range(w)] for _ in range(h)]
    
    for atyp, coords in an_coord:
        for x,y in coords:
            grid_ps[y][x] = atyp
    if antinode_coords is not None:
        for x,y in antinode_coords:
            c = grid_ps[y][x]
            if c == '.':
                grid_ps[y][x] = "#"
            else:
                grid_ps[y][x] = f"{atyp}'"
    for l in grid_ps:
        print(''.join(l))

In [190]:
dat = exmp
an_coord, grid = format_input(dat)
print_grid(an_coord, grid)

............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [194]:
atyp, coords = an_coord[0]
print(atyp, coords)

A [(6, 5), (8, 8), (9, 9)]


In [197]:
combs = generate_antenna_combinations(coords)
combs

[((6, 5), (8, 8)), ((6, 5), (9, 9)), ((8, 8), (9, 9))]

In [199]:
ans = [co for cb in combs for co in get_coord_antinodes(cb[0], cb[1])]
ans

[(4, 2), (10, 11), (3, 1), (7, 7), (10, 10)]

In [200]:
print_grid(an_coord, grid, ans)

............
...#....0...
....#0......
.......0....
....0.......
......A.....
............
.......#....
........A...
.........A..
..........#.
..........#.


looks good for A

In [202]:
atyp, coords = an_coord[1]
print(atyp, coords)

0 [(8, 1), (5, 2), (7, 3), (4, 4)]


In [203]:
combs = generate_antenna_combinations(coords)
combs

[((8, 1), (5, 2)),
 ((8, 1), (7, 3)),
 ((8, 1), (4, 4)),
 ((5, 2), (7, 3)),
 ((5, 2), (4, 4)),
 ((7, 3), (4, 4))]

In [204]:
ans = [co for cb in combs for co in get_coord_antinodes(cb[0], cb[1])]
ans

[(5, 0),
 (8, 3),
 (8, 5),
 (8, 7),
 (3, 1),
 (9, 4),
 (4, 0),
 (5, 6),
 (4, 2),
 (7, 5)]

In [205]:
print_grid(an_coord, grid, ans)

....##......
...#....0...
....#0......
.......0#...
....0....#..
......A##...
.....#......
........#...
........A...
.........A..
............
............


false

In [206]:
atyp, coords = an_coord[1]
print(atyp, coords)
combs = generate_antenna_combinations(coords)
print(combs)

0 [(8, 1), (5, 2), (7, 3), (4, 4)]
[((8, 1), (5, 2)), ((8, 1), (7, 3)), ((8, 1), (4, 4)), ((5, 2), (7, 3)), ((5, 2), (4, 4)), ((7, 3), (4, 4))]


In [207]:
cb = combs[0]
print(cb)
get_coord_antinodes(cb[0], cb[1])

((8, 1), (5, 2))


[(5, 0), (8, 3)]

In [213]:
## order antennas from ascending y then ascending x
an_sort = sorted(cb, key=lambda x: x[1])
c1, c2 = an_sort
print(c1, c2)
diff_x, diff_y = (abs(c1[0] - c2[0])), abs((c2[1] - c1[1]))
sign_x = 2*((c1[0] - c2[0]) > 0) - 1
an1 = (c1[0] + sign_x*diff_x, c1[1] - diff_y)
an2 = (c2[0] - sign_x*diff_x, c2[1] + diff_y)
print(an1, an2)

(8, 1) (5, 2)
(11, 0) (2, 3)


In [214]:
print_grid(an_coord, grid, [an1, an2])

...........#
........0...
.....0......
..#....0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


Now it looks good

In [215]:
def get_coord_antinodes(a1: Coord, a2: Coord, grid) -> list[Coord]:
    an_sort = sorted([a1, a2], key=lambda x: x[1])
    c1, c2 = an_sort
    diff_x, diff_y = (abs(c1[0] - c2[0])), abs((c2[1] - c1[1]))
    sign_x = 2*((c1[0] - c2[0]) > 0) - 1
    an1 = (c1[0] + sign_x*diff_x, c1[1] - diff_y)
    an2 = (c2[0] - sign_x*diff_x, c2[1] + diff_y)
    out = [an1, an2]

    w, h = len(grid[0]), len(grid)
    out = [(x,y) for (x,y) in out if x >= 0 and x < w and y >= 0 and y < h]
    return out

In [230]:
dat = exmp
an_coord, grid = format_input(dat)
combs = generate_antenna_combinations(coords)
combs
cb = combs[5]
print(cb)
ans = get_coord_antinodes(cb[0], cb[1], grid)
print(ans)
print_grid(an_coord, grid, ans)

((7, 3), (4, 4))
[(10, 2), (1, 5)]
............
........0...
.....0....#.
.......0....
....0.......
.#....A.....
............
............
........A...
.........A..
............
............


In [236]:
dat = exmp 
ants, grid = format_input(dat)
combs = [cb for _,coords in ants for cb in generate_antenna_combinations(coords)]
antinodes = set([co for cb in combs for co in get_coord_antinodes(cb[0], cb[1], grid)])
print_grid(ants, grid, antinodes)

......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....0'.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.


In [237]:
def sol1(dat: str) -> int: 
    ants, grid = format_input(dat)
    combs = [cb for _,coords in ants for cb in generate_antenna_combinations(coords)]
    antinodes = set([co for cb in combs for co in get_coord_antinodes(cb[0], cb[1], grid)])
    return len(antinodes)

In [239]:
dat = exmp
sol1(dat)

14

In [240]:
dat = data
sol1(dat)

359

summary 

In [243]:
Coord = tuple[int, int]
Grid = list[list[str]]

def format_input(dat: str) -> tuple[list[tuple[str, Coord]], Grid]:
    grid = [[c for c in l] for l in dat.splitlines()]
    all_antennas = [(c,(x,y)) for y,l in enumerate(grid) for x,c in enumerate(l) if c not in ('.', '\n')]
    antenna_types = set([c for c,_ in all_antennas])
    out = [(a, [co for c,co in all_antennas if c == a]) for a in antenna_types]
    return out, grid


def generate_antenna_combinations(antenna_coords: list[Coord]) -> list[tuple[Coord, Coord]]:
    from itertools import combinations
    return list(combinations(antenna_coords, 2))


def get_coord_antinodes(a1: Coord, a2: Coord, grid) -> list[Coord]:
    an_sort = sorted([a1, a2], key=lambda x: x[1])
    c1, c2 = an_sort
    diff_x, diff_y = (abs(c1[0] - c2[0])), abs((c2[1] - c1[1]))
    sign_x = 2*((c1[0] - c2[0]) > 0) - 1
    an1 = (c1[0] + sign_x*diff_x, c1[1] - diff_y)
    an2 = (c2[0] - sign_x*diff_x, c2[1] + diff_y)
    out = [an1, an2]
    w, h = len(grid[0]), len(grid)
    out = [(x,y) for (x,y) in out if x >= 0 and x < w and y >= 0 and y < h]
    return out


def sol1(dat: str, disp: bool = False) -> int: 
    ants, grid = format_input(dat)
    combs = [cb for _,coords in ants for cb in generate_antenna_combinations(coords)]
    antinodes = set([co for cb in combs for co in get_coord_antinodes(cb[0], cb[1], grid)])
    if disp:
        print_grid(ants, grid, antinodes)
    return len(antinodes)


def print_grid(an_coord, grid, antinode_coords = None):
    w, h = len(grid[0]), len(grid)
    grid_ps = [['.' for _ in range(w)] for _ in range(h)]
    
    for atyp, coords in an_coord:
        for x,y in coords:
            grid_ps[y][x] = atyp
    if antinode_coords is not None:
        for x,y in antinode_coords:
            c = grid_ps[y][x]
            if c == '.':
                grid_ps[y][x] = "#"
            else:
                grid_ps[y][x] = f"{atyp}'"
    for l in grid_ps:
        print(''.join(l))

In [248]:
dat = exmp
sol1(dat, True)

......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....0'.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.


14

In [249]:
dat = data
sol1(dat, True)

...O.#...0..#.#..........................v'..v'.....
O.......#.o....w#.T.....................#...p.....
.......#.........#w..........oM..#.............#..
.......#........#....##...................#..Y.##.
o.............T.....#............##.......z.....pk
..#................................#.z..Y...#t#F..
...........T........#...........#....#F.......Y...
...#......#........A........##..z..#k#.M........#.
#...O...#.....j....w#...#................M........
..........w....T#....#.....#......v'..k.....#...#.#
.#...........v'........##...#.......#..#....F.....t
....#....#..........#.A......#...#..F.#..E........
...........#......##.S.#.....#.A...#............#.
.P.....#.......................#....#...#...#.....
.#.......#......#...#...v'#..v'......#......x.##v'##.
........#...j.......#..#.#........#...t#.....#..x.
..#..#..#...#....##....j....#.#...#...#..#......#v'
.....#......#..#.........##.......#.E......#.v'.#..
.............P.......E.....#..#...........##......
......#......#.v'..5#.

359

## part 2

In [250]:
await rephrase_pb(DAY, YEAR, 2)

Let me rephrase the problem:

**Antinodes** occur in two cases:
1. When a location is exactly in line (horizontally, vertically, or diagonally) with at least two antennas of the same frequency
2. At the position of each antenna (unless it's the only antenna of its frequency)
 
You need to **count the total number of unique antinode positions** within the map boundaries.
 
Example input map looks like this:
```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```
Where:
- **T** represents antennas of the same frequency
- **#** represents other elements on the map
 
The goal is to find all positions that satisfy either of the antinode conditions and count them.

this requires changing the get_coord_antinodes function, to add all multiples of the distance

In [254]:
def get_coord_antinodes_2(a1: Coord, a2: Coord, grid) -> list[Coord]:
    an_sort = sorted([a1, a2], key=lambda x: x[1])
    c1, c2 = an_sort
    diff_x, diff_y = (abs(c1[0] - c2[0])), abs((c2[1] - c1[1]))
    sign_x = 2*((c1[0] - c2[0]) > 0) - 1

    w, h = len(grid[0]), len(grid)

    anti_coords = [a1, a2]

    xi,yi = c1[0], c1[1]
    while True:
        xi = xi + sign_x*diff_x
        yi = yi - diff_y
        if xi >= 0 and xi < w and yi >= 0 and yi < h:
            anti_coords.append((xi, yi))
        else:
            break
    
    xj,yj = c2[0], c2[1]
    while True:
        xj = xj - sign_x*diff_x
        yj = yj + diff_y
        if xj >= 0 and xj < w and yj >= 0 and yj < h:
            anti_coords.append((xj, yj))
        else:
            break

    return anti_coords


def sol2(dat: str, disp: bool = False) -> int: 
    ants, grid = format_input(dat)
    combs = [cb for _,coords in ants for cb in generate_antenna_combinations(coords)]
    antinodes = set([co for cb in combs for co in get_coord_antinodes_2(cb[0], cb[1], grid)])
    if disp:
        print_grid(ants, grid, antinodes)
    return len(antinodes)

In [255]:
dat = exmp 
sol2(dat)

34

In [256]:
dat = data 
sol2(dat)

1293

## take aways

- what is the nudge in this problem set, if any ? Part 2 might have been encouraging recursion ?